In [1]:
from genecover import *
import numpy as np
import scanpy as sc
import pandas as pd
import os 

# Marker Gene Selection from DLPFC Sample #151673

### Load Dataset

In [ ]:
data_dir = "..\\data\\DLPFC"
file_folder = "151673"
path = os.path.join(data_dir, file_folder)
adata = sc.read_visium(path,count_file = "filtered_feature_bc_matrix.h5",load_images=True)
adata.var_names_make_unique()
df_meta = pd.read_csv(os.path.join(path, "metadata.tsv"), sep='\t')
df_meta_layer = df_meta['layer_guess']
adata.obs['ground_truth'] = df_meta_layer.values
adata = adata[~pd.isnull(adata.obs['ground_truth'])]
sc.pp.filter_genes(adata, min_cells=100)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
orig_gene = adata.var.index.values

C:\Users\An Wang\AppData\Local\Temp\ipykernel_8748\1400016560.py:4: FutureWarning: Use `squidpy.read.visium` instead.
  adata = sc.read_visium(path,count_file = "filtered_feature_bc_matrix.h5",load_images=True)
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\scanpy\preprocessing\_simple.py:287: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var["n_cells"] = number


### Computing Correlation Matrix

In [8]:
corr_mat = gene_gene_correlation(adata.X.toarray())

### GeneCover via Combinatorial Optimization (Gurobi Solver)

In [ ]:
# Obtain 100 marker genes 
genecover_markers = GeneCover(num_marker=100, corr_mat=corr_mat, w=np.ones(corr_mat.shape[1]), greedy=False)
print("GeneCover markers: \n", orig_gene[genecover_markers])

Best Gap:  0
Best Epsilon:  0.1311767578125
GeneCover markers: 
 ['MARCKSL1' 'SYNC' 'ATP1A1' 'ATP1B1' 'NME7' 'CNTN2' 'HPCAL1' 'VSNL1'
 'MDH1' 'PPP3R1' 'CTNNA2' 'TMSB10' 'IGKC' 'MAL' 'ERMN' 'LRP2' 'GAD1'
 'CHN1' 'MOBP' 'CCK' 'AC106707.1' 'CLDN11' 'FAM131A' 'LDB2' 'UCHL1' 'SPP1'
 'BBS7' 'HHIP' 'TMEM144' 'GPM6A' 'SLC1A3' 'ENC1' 'EDIL3' 'CXCL14' 'GABRB2'
 'MOG' 'ACTB' 'NDUFA4' 'RAPGEF5' 'AQP1' 'PHKG1' 'YWHAG' 'SLC26A4-AS1'
 'GJB1' 'NAP1L2' 'NEFM' 'NEFL' 'STMN2' 'CALB1' 'ENPP2' 'DIRAS2' 'CERCAM'
 'OLFM1' 'PTGDS' 'SAA1' 'FOLH1' 'MYRF' 'SCGB2A2' 'CARNS1' 'NRGN' 'NKX6-2'
 'NELL2' 'KRT8' 'KCNC2' 'SYT1' 'CUX2' 'CABP1' 'AACS' 'RTN1' 'HSPA2'
 'PPP4R4' 'GLDN' 'HBA2' 'SLC5A11' 'AC009133.1' 'PLLP' 'NDRG4' 'CALB2'
 'KRT19' 'CNP' 'GFAP' 'NSF' 'ANKRD40' 'AATK' 'MBP' 'CHGB' 'SNAP25' 'CST3'
 'BCAS1' 'NDUFA7' 'PPP1R14A' 'CALM3' 'RSPH14' 'NEFH' 'YWHAH' 'C21orf91'
 'OLIG1' 'PCP4' 'MT-CO1' 'MT-CO2']


### GeneCover via Greedy Heuristics (Don't Need Gurobi)

In [ ]:
#obtain 100 marker genes via greedy heuristics
genecover_markers_greedy = GeneCover(num_marker=100, corr_mat=corr_mat, w=np.ones(corr_mat.shape[1]), greedy=True)
print("GeneCover markers: \n", orig_gene[genecover_markers_greedy])

Best Gap:  0
Best Epsilon:  0.136181640625
GeneCover markers: 
 ['SNAP25' 'ERMN' 'ENC1' 'VSNL1' 'MAG' 'SCGB2A2' 'ATP1B1' 'RTN1' 'PTGDS'
 'OLFM1' 'MOG' 'MBP' 'FABP4' 'IGKC' 'CST3' 'CARNS1' 'GAD1' 'TMEM144'
 'KRT19' 'MT-CO1' 'NME7' 'UCHL1' 'PPP1R14A' 'CHN1' 'CLDN11' 'HBB' 'HSPA2'
 'CALB2' 'EDIL3' 'YWHAG' 'GJB1' 'NEFL' 'ENPP2' 'KRT8' 'RNASE1' 'SLC5A11'
 'GFAP' 'YWHAH' 'SYNC' 'RPL5' 'ATP1A1' 'HPCAL1' 'FAM84A' 'PPP3R1' 'TMSB10'
 'MOBP' 'LDB2' 'LIMCH1' 'SPP1' 'HHIP' 'SV2C' 'SLC12A2' 'CXCL14' 'TUBB2A'
 'ACTB' 'RAPGEF5' 'AQP1' 'PHKG1' 'SEMA3E' 'CHRDL1' 'LAMP2' 'NEFM' 'CALB1'
 'DIRAS2' 'SLC44A1' 'STXBP1' 'CERCAM' 'ABCA2' 'FOLH1' 'FTH1' 'THY1'
 'HSPA8' 'NRGN' 'NKX6-2' 'VAMP1' 'SYT1' 'CABP1' 'AACS' 'HTR2A' 'SLAIN1'
 'HS6ST3' 'GLDN' 'IQCK' 'GPRC5B' 'AC009133.1' 'PLLP' 'NDRG4' 'RPL26'
 'LRRC75A' 'KRT17' 'NSF' 'ANKRD40' 'SEPT4' 'AATK' 'CHGB' 'LAMP5' 'NEFH'
 'OLIG1' 'S100B' 'MT-CO3']


### Iterative GeneCover via Combinatorial Optimization

In [14]:
# Obtain 200 marker genes via Iterative GeneCover with an incremental size of 100 and two iterations
genecover_markers_iterative = Iterative_GeneCover(incremental_sizes=[100,100], corr_mat=corr_mat, w=np.ones(corr_mat.shape[1]), greedy=False)

Iteration 1
Best Gap:  0
Best Epsilon:  0.1311767578125
Iteration  2
Best Gap:  0
Best Epsilon:  0.1367919921875


### Iterative GeneCover via Greedy Heuristics

In [17]:
# Obtain 200 marker genes via Iterative GeneCover (Greedy Heuristics) with an incremental size of 100 and two iterations
genecover_markers_iterative = Iterative_GeneCover(incremental_sizes=[100,100], corr_mat=corr_mat, w=np.ones(corr_mat.shape[1]), greedy=True)

Iteration 1
Best Gap:  0
Best Epsilon:  0.136181640625
Iteration  2
Best Gap:  0
Best Epsilon:  0.14765625


# Marker Gene Selection Across Samples

### Load the datasets (Sample #1515067, #151669, #151673)

In [28]:
Adata = {}
data_dir = "..\\data\\DLPFC Full Samples"
file_names = os.listdir(data_dir)
across_samples_file_names = file_names[:3]
annotation_names = file_names[3:]
for j,  file in enumerate(across_samples_file_names):
    adata = sc.read_10x_h5(os.path.join(data_dir, file))
    adata.var_names_make_unique()
    layer_annotation = pd.read_csv(os.path.join(data_dir, annotation_names[j]), index_col=0)
    assert np.all(adata.obs.index.values == layer_annotation.index.values)
    adata.obs = layer_annotation
    sc.pp.filter_genes(adata, min_cells=100)
    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, n_top_genes=10000)
    adata = adata[:, adata.var.highly_variable]
    Adata[file.split("_")[0]] = adata

for j, file in enumerate(across_samples_file_names):
    if j == 0: 
        genes_across_samples = Adata[file.split("_")[0]].var_names.values
    else:
        genes_across_samples = np.intersect1d(genes_across_samples, Adata[file.split("_")[0]].var_names.values)

for key in across_samples_file_names:
    Adata[key.split("_")[0]] = Adata[key.split("_")[0]][:, genes_across_samples]

c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
c:\Users\An Wang\.conda\envs\vae-spatial\lib\site-packages\anndata\_core\anndata.py:1758: UserWarning: Varia

### Compute Correlation Matrices for Every Sample

In [29]:
corr_mat_combine_across_samples = gene_gene_correlation([Adata[key.split("_")[0]].X.toarray() for key in across_samples_file_names])

### GeneCover Marker Selection via Combinatorial Optimization Across Samples

In [ ]:
# Obtain 100 marker genes across samples
genecover_marker_across_samples = GeneCover(num_marker=100, corr_mat = corr_mat_combine_across_samples, w = np.ones(corr_mat_combine_across_samples.shape[1]), greedy = False)
print("GeneCover markers: \n", genes_across_samples[genecover_marker_across_samples])

Best Gap:  0
Best Epsilon:  0.10332946777343749
GeneCover markers: 
 ['AC005944.1' 'AC009133.1' 'AC092069.1' 'ACTB' 'ALDOA' 'AP3B2' 'ATP1A1'
 'ATP1B1' 'ATP6V1B2' 'B2M' 'BASP1' 'CARTPT' 'CCK' 'CHN1' 'CLSTN2' 'CLSTN3'
 'CPLX1' 'CRYM' 'CST3' 'DIRAS2' 'DYNC1I1' 'ENC1' 'EPHA4' 'FABP4' 'G0S2'
 'GABRA1' 'GABRG2' 'GAD1' 'GAP43' 'GFAP' 'GPM6A' 'HBB' 'HOPX' 'IGHA1'
 'IGLC2' 'KLK6' 'KRT17' 'KRT19' 'KRT8' 'LMO4' 'MAGEE1' 'MAP1B' 'MBP'
 'MDH1' 'MICAL2' 'MOAP1' 'MOBP' 'MT-ND2' 'MT3' 'NDRG4' 'NEFH' 'NEFL'
 'NEFM' 'NELL2' 'NME7' 'NRGN' 'NSF' 'OAT' 'OLFM1' 'PAK1' 'PDP1' 'PHKG1'
 'PLIN1' 'PLP1' 'PPP3R1' 'RAB3A' 'REEP2' 'RPLP1' 'RTN1' 'RTN3' 'RTN4'
 'SCGB2A2' 'SCN1A' 'SH3BGRL2' 'SLC17A7' 'SLC1A2' 'SLC24A2' 'SLC39A10'
 'SNAP25' 'SNCB' 'SPP1' 'STMN1' 'STMN2' 'SYNC' 'SYNGR1' 'SYT1' 'SYT4'
 'TFF1' 'TMEM38A' 'TMEM59L' 'TMSB10' 'TUBA1A' 'TUBA1B' 'TUBB2A' 'UCHL1'
 'VPS35' 'VSNL1' 'WDR47' 'WDR7' 'YWHAG']


### GeneCover Marker Selection via Greedy Heuristics Across Samples

In [ ]:
# Obtain 100 marker genes across samples via greedy heuristics
genecover_marker_across_samples_greedy = GeneCover(num_marker=100, corr_mat = corr_mat_combine_across_samples, w = np.ones(corr_mat_combine_across_samples.shape[1]), greedy = True)
print("GeneCover markers: \n", genes_across_samples[genecover_marker_across_samples_greedy])

Best Gap:  0
Best Epsilon:  0.11249999999999999
GeneCover markers: 
 ['SNAP25' 'PLP1' 'ENC1' 'SYT1' 'SCGB2A2' 'VSNL1' 'NRGN' 'CST3' 'ATP1B1'
 'TUBA1B' 'RTN1' 'MOG' 'UCHL1' 'SAA1' 'NEFL' 'TMSB10' 'ERMN' 'GFAP' 'IGKC'
 'KRT19' 'ACTB' 'MBP' 'OLFM1' 'MAP1B' 'CLDN11' 'SLC24A2' 'TUBB2A' 'MDH1'
 'CARNS1' 'CHN1' 'CLSTN2' 'DIRAS2' 'MT-ND1' 'PTGDS' 'HBA1' 'STMN2'
 'NDUFA4' 'RTN3' 'MAG' 'NEFM' 'STMN1' 'GABRB2' 'HSPA8' 'RPLP1' 'SLC1A2'
 'AC092069.1' 'ATP6V1B2' 'CPNE4' 'EFHD2' 'NSF' 'PCP4' 'SPP1' 'YWHAG' 'CCK'
 'CNDP1' 'CNP' 'ENPP2' 'GABRA1' 'HOPX' 'HPCAL1' 'MICAL2' 'PHKG1' 'TBR1'
 'GAP43' 'LMO4' 'MT3' 'RAB3A' 'SERPINI1' 'BASP1' 'CDC37' 'COX6C' 'CPB1'
 'CRYAB' 'ETS2' 'FKBP1A' 'KCNK1' 'KRT18' 'KRT8' 'LRRC75A' 'MARCKSL1'
 'NARS' 'NCOA7' 'NDFIP1' 'NREP' 'PFKP' 'PRKCE' 'PTPRN' 'PVALB' 'RAB3C'
 'RPS12' 'S100B' 'SCG5' 'SH3GL2' 'SLC17A7' 'SNCA' 'SNCG' 'TF' 'TFF1'
 'TMEM59L' 'YWHAB']


### Iterative GeneCover via Combinatorial Optimization Across Samples

In [34]:
# Obtain 200 marker genes via Iterative GeneCover with an incremental size of 100 and two iterations
genecover_marker_across_samples = genecover_markers_iterative = Iterative_GeneCover(incremental_sizes=[100,100], corr_mat=corr_mat_combine_across_samples, w=np.ones(corr_mat_combine_across_samples.shape[1]), greedy=False)

Iteration 1
Best Gap:  0
Best Epsilon:  0.10332946777343749
Iteration  2
Best Gap:  0
Best Epsilon:  0.10310058593749999


In [ ]:
# Obtain 200 marker genes via Iterative GeneCover with an incremental size of 100 and two iterations
genecover_marker_across_samples_greedy = genecover_markers_iterative = Iterative_GeneCover(incremental_sizes=[100,100], corr_mat=corr_mat_combine_across_samples, w=np.ones(corr_mat_combine_across_samples.shape[1]), greedy=True)

Iteration 1
Best Gap:  0
Best Epsilon:  0.11249999999999999
Iteration  2
